In [1]:
import argparse
import glob
import json
import os
import re
from importlib import import_module
from pathlib import Path

import numpy as np
import torch
from torch.utils.data import Subset
from torch.utils.tensorboard import SummaryWriter
# from utils import label_accuracy_score, seed_everything, seed_worker, copyblob
import albumentations as A
from albumentations.pytorch import ToTensorV2

# from loss import create_criterion

import time
# from utils import load_model

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("device: ", device)

device:  cuda


In [3]:
# train.json / validation.json / test.json 디렉토리 설정
# train_path = '/opt/ml/input/data/jaeyoung_test_psuedo.json'
train_path = '/opt/ml/input/data/train_all.json'

In [4]:
# collate_fn needs for batch
def collate_fn(batch):
  return tuple(zip(*batch))

In [5]:
train_transform = A.Compose([
    ToTensorV2()
])

In [6]:
from pycocotools.coco import COCO
import torchvision
import torchvision.transforms as transforms
import albumentations as A
import json
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import cv2
import os

In [7]:
dataset_path = '/opt/ml/input/data/'
class DetectionDataset(Dataset):
    """COCO format"""
    num_classes = 12
    def __init__(self, data_dir, category_names, mode = 'train', transform = None, cutmix=False):
        super().__init__()
        self.mode = mode
        self.transform = transform
        self.coco = COCO(data_dir)
        self.category_names = ['Backgroud', 'UNKNOWN', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']
        self.mean_std = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        self.image_num = len(self.coco.getImgIds())
        self.cutmix = cutmix
        
    def __getitem__(self, index: int):
        # # cutmix용 index2 (범위안에서 랜덤으로 한장 뽑기), 근데 random 보다 class 별로 확률을 주는 것도 좋을 듯
        index2 = np.random.randint(0, self.image_num)

        # dataset이 index되어 list처럼 동작
        image_id = self.coco.getImgIds(imgIds=index)
        image_infos = self.coco.loadImgs(image_id)[0]
        
        # cv2 를 활용하여 image 불러오기
        images = cv2.imread(os.path.join(dataset_path, image_infos['file_name']))
        images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB).astype(np.float32)
        images /= 255.0
        
        if (self.mode == 'train' or self.mode == 'val'):
            # cutmix용 image2 load
            if self.cutmix:
                cutmix_p = np.random.rand()
                if cutmix_p > 0.7:
                    image_id2 = self.coco.getImgIds(imgIds=index2)
                    image_infos2 = self.coco.loadImgs(image_id2)[0]
                    images2 = cv2.imread(os.path.join(dataset_path, image_infos2['file_name']))
                    images2 = cv2.cvtColor(images2, cv2.COLOR_BGR2RGB).astype(np.float32)
                    images2 /= 255.0

            ann_ids = self.coco.getAnnIds(imgIds=image_infos['id']) # annotation ID
            anns = self.coco.loadAnns(ann_ids) # annotation 불러오기
            if self.cutmix: # cutmix 설정을 했다면
                if cutmix_p > 0.7: # cutmix를 설정할 확률이 0.7보다 크다면 
                    ann_ids2 = self.coco.getAnnIds(imgIds=image_infos2['id']) # 해당 이미지의 annotation id를 가져온다
                    anns2 = self.coco.loadAnns(ann_ids2) # 그 id에 해당하는 annotation을 가져온다.

            # Load the categories in a variable
            cat_ids = self.coco.getCatIds()
            cats = self.coco.loadCats(cat_ids) # 해당 category id를 가져온다. 예를 들면 종량제, 건전지..
            # print("cats: ", cats)
            # cats:  [{'id': 0, 'name': 'UNKNOWN', 'supercategory': 'UNKNOWN'}, 
            # {'id': 1, 'name': 'General trash', 'supercategory': 'General trash'}, 
            # {'id': 2, 'name': 'Paper', 'supercategory': 'Paper'}, 
            # {'id': 3, 'name': 'Paper pack', 'supercategory': 'Paper pack'}, 
            # {'id': 4, 'name': 'Metal', 'supercategory': 'Metal'}, 
            # {'id': 5, 'name': 'Glass', 'supercategory': 'Glass'}, 
            # {'id': 6, 'name': 'Plastic', 'supercategory': 'Plastic'}, 
            # {'id': 7, 'name': 'Styrofoam', 'supercategory': 'Styrofoam'},
            # {'id': 8, 'name': 'Plastic bag', 'supercategory': 'Plastic bag'}, 
            # {'id': 9, 'name': 'Battery', 'supercategory': 'Battery'}, 
            # {'id': 10, 'name': 'Clothing', 'supercategory': 'Clothing'}]

            # masks : size가 (height x width)인 2D
            # 각각의 pixel 값에는 "category id + 1" 할당
            # Background = 0
            res = COCO()
            if 'bbox' in anns[0] and not anns[0]['bbox']==[]:
              # Unknown = 1, General trash = 2, ... , Cigarette = 11
              for id, ann in enumerate(anns): #해당 anns_id에 해당하는 anns정보를 돌면서
                bb = ann['bbox'] # bbox: 객체가 존재하는 박스의 좌표 (xmin, ymin, w, h)

                xmin, ymin, w, h = bb
                xmin, ymin, xmax, ymax = xmin, ymin, xmin+w, ymin+h
                rectangle = [xmin, ymin, xmax, ymax]

                # ann['area'] = bb[2]*bb[3]
                ann['id'] = id+1
                # ann['iscrowd'] = 0
                # print("bb: ",bb)
              if self.transform is not None:
                transformed = self.transform(image=images)
                images = transformed["image"]
                images = transforms.Normalize(*self.mean_std)(images)
            
            return images, rectangle, image_infos

        if self.mode == 'test':
            # transform -> albumentations 라이브러리 활용
            if self.transform is not None:
                transformed = self.transform(image=images)
                images = transformed["image"]
                images = transforms.Normalize(*self.mean_std)(images)
            
            return images, image_infos
    
    
    def __len__(self) -> int:
        # 전체 dataset의 size를 return
        return len(self.coco.getImgIds())

In [8]:
category_names = ['Backgroud', 'UNKNOWN', 'General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing']
train_dataset = DetectionDataset(data_dir=train_path, category_names=category_names, mode='train', transform=train_transform, cutmix=False)

loading annotations into memory...
Done (t=9.00s)
creating index...
index created!


In [9]:
# DataLoader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                        batch_size=8,
                                        shuffle=True,
                                        num_workers=4,
                                        collate_fn=collate_fn,
                                        drop_last=True,
                                        )

In [10]:
# prepare data loaders
dataset = dataset if isinstance(dataset, (list, tuple)) else [dataset]
if 'imgs_per_gpu' in cfg.data:
    logger.warning('"imgs_per_gpu" is deprecated in MMDet V2.0. '
                   'Please use "samples_per_gpu" instead')
    if 'samples_per_gpu' in cfg.data:
        logger.warning(
            f'Got "imgs_per_gpu"={cfg.data.imgs_per_gpu} and '
            f'"samples_per_gpu"={cfg.data.samples_per_gpu}, "imgs_per_gpu"'
            f'={cfg.data.imgs_per_gpu} is used in this experiments')
    else:
        logger.warning(
            'Automatically set "samples_per_gpu"="imgs_per_gpu"='
            f'{cfg.data.imgs_per_gpu} in this experiments')
    cfg.data.samples_per_gpu = cfg.data.imgs_per_gpu

data_loaders = [
    build_dataloader(
        ds,
        cfg.data.samples_per_gpu,
        cfg.data.workers_per_gpu,
        # cfg.gpus will be ignored if distributed
        len(cfg.gpu_ids),
        dist=distributed,
        seed=cfg.seed) for ds in dataset
]

NameError: name 'dataset' is not defined

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as img 

In [ ]:
def imshow(input, title):
    # torch.Tensor를 numpy 객체로 변환
    input = input.numpy().transpose((1, 2, 0))
    # 이미지 정규화 해제하기
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    input = std * input + mean
    input = np.clip(input, 0, 1)
    # 이미지 출력
    plt.imshow(input)
    plt.title(title)
    plt.show()

In [ ]:
def bbshow(images, rectangle, image_infos):
  # image = img.imread(dataset_path+'/'+image_infos[0]['file_name'])
  # imshow(images[0], '')
  for i in range(len(images)):
    x1, y1, x2, y2 = rectangle[i]
    print(images[i].shape)
    print(rectangle[i])
    cv2.rectangle(images[i].numpy().transpose((1, 2, 0)), (int(x1), int(y1)), (int(x2), int(y2)), (255,0,0), 5)
    imshow(images[i], '')
    plt.show()

In [ ]:
print(len(train_loader))

In [ ]:
iterator = iter(train_loader)
images, rectangle, image_infos = next(iterator)
print(image_infos)
# out = torchvision.utils.make_grid(images)
bbshow(images, rectangle, image_infos)